In [1]:
import json
import random
import numpy as np


# simple hack to support import module from parent directory
import sys
sys.path.append('../')

from instruct_llama.utils import build_prompt_completion

DEFAULT_SYSTEM_PROMPT = {
    "role": "system",
    "content": "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Please ensure that your responses are socially unbiased and correct. If you don't know the answer to a question, please don't share false information.",
}

# build the training data using dialog style
DEFAULT_DIALOG = [DEFAULT_SYSTEM_PROMPT]

In [2]:
train_file = "/home/michael/InstructLLaMA/raw_data/msc_dialogue/session_2/train.txt"

content = []

with open(train_file, "r", encoding="utf-8") as file:
    for line in file:
        content.append(json.loads(line.strip()))

In [3]:
dialogs = []


for item in content:
    conversation = item["dialog"]

    assert len(conversation) > 2, f"dialog length: {len(conversation)}"

    if len(conversation) % 2 != 0:
        conversation = conversation[:-1]

    assert len(conversation) % 2 == 0, f"dialog length: {len(conversation)}"

    first_id = conversation[0]["id"]
    second_id = conversation[1]["id"]

    assert first_id != second_id

    if not (
        all([msg["id"] == first_id for msg in conversation[0::2]])
        and all([msg["id"] == second_id for msg in conversation[1::2]])
    ):
        continue

    dialog = [{"role": "user" if i % 2 == 0 else "assistant", "content": item["text"]} for i, item in enumerate(conversation)]

    assert all([msg["role"] == "user" for msg in dialog[0::2]]) and all([msg["role"] == "assistant" for msg in dialog[1::2]])

    dialogs.append(dialog)



In [4]:
dialog_turns = [len(dialog) for dialog in dialogs]

print(np.mean(dialog_turns))
print(np.min(dialog_turns))
print(np.max(dialog_turns))

11.921357850070722
6
12


In [8]:
def sample_dialogs(n=5):
    random.shuffle(dialogs)

    for dialog in dialogs[:n]:
        for i in range(0, len(dialog)-1, 2):
            print(f"Question: {dialog[i]['content']}")
            print(f"Answer: {dialog[i+1]['content']}")

        print("\n"+"-"*60+"\n")

In [11]:
sample_dialogs()

Question: I am glad that I got a new job.
Answer: Oh you did, where? I'm glad you got one.
Question: Thanks! I'm starting on Monday as a barista at Starbucks! Maybe I can make you a coffee some day.
Answer: I would love that. I love iced coffee so much.
Question: I bet you drink a lot of coffee with 3 kids! How old are they?
Answer: Haha yes I have to or i'll go crazy. My girl is five and the boys are seven and ten. They are growing up so fast.
Question: I can imagine. I feel like it was just last week that I brought home Toby from the shelter. Now he's fat and old and just lays around staring at birds.
Answer: Haha that's such a mood!! I bet he's the coolest. 
Question: He's a pretty great cat. Do you have any pets? I bet your kids would love a new kitten or puppy!
Answer: I've been thinking about it for a while. I was thinking maybe a dog or a fish. But hearing about your cat I kind of want one for my kids now haha.
Question: Cats are the perfect pet, especially with kids. They are l